In [ ]:
import os
import cv2
from time import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, models, transforms
from sklearn.model_selection import train_test_split
import torchvision


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/DL Project/TrafficSign DataSet.zip', 'r') as zip_ref:
    zip_ref.extractall('./')


In [ ]:
def ResizeImage(image: np.ndarray, newWidth: int, newHeight: int) -> tuple:
    resizedImage = cv2.resize(
        image, (newWidth, newHeight), interpolation=cv2.INTER_LINEAR)
    return resizedImage

In [ ]:
def LoadDataSet(dataSetFolderPath: str) -> tuple:
    images = []
    annotations = []
    annotationsFilePath = dataSetFolderPath+"/tafficSignAnnotations.csv"
    annotationsDataFrame = pd.read_csv(annotationsFilePath, sep=", ")
    print(annotationsDataFrame.columns)
    uniqueSigns = annotationsDataFrame['Catoregy'].unique().tolist()
    for index, row in annotationsDataFrame[1:].iterrows():
        image = cv2.imread("."+row[1])
        images.append(image)
        annotations.append(uniqueSigns.index(row[0]))
    del annotationsDataFrame
    return images, annotations, len(uniqueSigns)


In [ ]:
def PreProcessDataSet(images: list, annotations: list, batchSize: int, resize: tuple) -> tuple:
    resizedImages = []
    newAnnotations = []
    for i, image in enumerate(images):
        label = annotations[i]
        resizedImage = ResizeImage(
            image, resize[0], resize[1])
        resizedImages.append(resizedImage)
        newAnnotations.append(label)

    X_train, X_val, y_train, y_val = train_test_split(
        resizedImages, newAnnotations, test_size=0.3, random_state=42)

    return X_train, X_val, y_train, y_val


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputData, label = self.data[idx]

        if self.transform:
            inputData = self.transform(inputData)
        inputData = torch.from_numpy(inputData).float()
        label = torch.tensor(label)
        return inputData, label

def CreateDataLoaders(X_train, X_val, y_train, y_val, batchSize):
    trainDataSet = []
    valDataSet = []
    for i in range(len(X_train)):
        trainDataSet.append((X_train[i], y_train[i]))

    for i in range(len(X_val)):
        valDataSet.append((X_val[i], y_val[i]))

    trainDataSet = CustomDataset(trainDataSet)
    valDataSet = CustomDataset(valDataSet)
    trainDataLoader = DataLoader(
        trainDataSet, batch_size=batchSize, shuffle=True, num_workers=4)
    valDataLoader = DataLoader(
        valDataSet, batch_size=batchSize, shuffle=False, num_workers=4)

    return trainDataLoader, valDataLoader


In [ ]:
def PredictSignBoard(model, dataLoader, device):
    predictions = []
    dataLoader = dataLoader.to(device)
    allLabels = []
    for i, (inputs, labels) in enumerate(dataLoader):
        inputs = inputs.permute(0, 3, 1, 2)
        with torch.no_grad():
            output = model(inputs)
            _, preds = torch.max(output, 1)
        predictions.append(preds.item())    
        allLabels.append(labels)
    return predictions, allLabels


In [ ]:
batchSize = 32
inputShape = (416, 416)
epochs = 100
numAnchors = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print("Using {} device".format(device))


In [ ]:
images, annotations, numClasses = LoadDataSet("./TrafficSign DataSet")

In [ ]:
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, numClasses)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
savedModels = [ modelName for modelName in os.listdir('/content/drive/MyDrive/DL Project/Trained Models/') if ('signPrediction' in modelName)]
savedModels.sort()
model.load_state_dict(torch.load('/content/drive/MyDrive/DL Project/Trained Models/'+savedModels[-1]))

In [ ]:
X_train, X_val, y_train, y_val = PreProcessDataSet(
    images, annotations, batchSize, inputShape)
del images
del annotations


In [ ]:
trainDataLoader, valDataLoader = CreateDataLoaders(
    X_train, X_val, y_train, y_val, batchSize)
del X_train
del y_train
del X_val
del y_val


In [ ]:
predictions, allLabels = PredictSignBoard(model, valDataLoader, device)